In [27]:
import sys
sys.path.append('../input/iterativestratification')

import numpy as np
import random
import pandas as pd
import os
import copy
import gc

from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt 
import seaborn as sns
import optuna

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_selection import VarianceThreshold, SelectKBest
from sklearn.multioutput import MultiOutputClassifier
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.pipeline import Pipeline
import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')

os.listdir('../input/lish-moa')

pd.set_option('max_columns', 2000)

In [28]:
ROOT = ".."
INPUT = "input"
LISH_MOA = "lish-moa"
NUM_OPTUNA_TRIAL = 50
NFOLDS = 2

# n_comp_GENES = 463
n_comp_GENES = 100
# n_comp_CELLS = 60
n_comp_CELLS = 70
VarianceThreshold_for_FS = 0.9
Dropout_Model = 0.25
QuantileTransformer_n_quantiles = 500 # instead of 100
print('n_comp_GENES', n_comp_GENES, 'n_comp_CELLS', n_comp_CELLS, 'total', n_comp_GENES + n_comp_CELLS)

n_comp_GENES 100 n_comp_CELLS 70 total 170


### It is recommended:
* **n_comp_GENES** smaller, 
* **n_comp_CELLS** more,
* **VarianceThreshold_for_FS** more, so that **train_features** is less.

## Read data

In [29]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')

test_features = pd.read_csv('../input/lish-moa/test_features.csv')
sample_submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

## FE & Data Preprocessing

In [30]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

### RankGauss

In [31]:
# RankGauss - transform to Gauss

for col in (GENES + CELLS):

    transformer = QuantileTransformer(n_quantiles=QuantileTransformer_n_quantiles,random_state=0, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

### Seed

In [32]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything(seed=42)

### PCA features

In [33]:
len(GENES)

772

In [34]:
# GENES

data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
data2 = (PCA(n_components=n_comp_GENES, random_state=42).fit_transform(data[GENES]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp_GENES)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp_GENES)])

train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [35]:
len(CELLS)

100

In [36]:
# CELLS

data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
data2 = (PCA(n_components=n_comp_CELLS, random_state=42).fit_transform(data[CELLS]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp_CELLS)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp_CELLS)])

train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [37]:
train_features.shape

(23814, 1046)

In [38]:
train_features.head(5)

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,g-10,g-11,g-12,g-13,g-14,g-15,g-16,g-17,g-18,g-19,g-20,g-21,g-22,g-23,g-24,g-25,g-26,g-27,g-28,g-29,g-30,g-31,g-32,g-33,g-34,g-35,g-36,g-37,g-38,g-39,g-40,g-41,g-42,g-43,g-44,g-45,g-46,g-47,g-48,g-49,g-50,g-51,g-52,g-53,g-54,g-55,g-56,g-57,g-58,g-59,g-60,g-61,g-62,g-63,g-64,g-65,g-66,g-67,g-68,g-69,g-70,g-71,g-72,g-73,g-74,g-75,g-76,g-77,g-78,g-79,g-80,g-81,g-82,g-83,g-84,g-85,g-86,g-87,g-88,g-89,g-90,g-91,g-92,g-93,g-94,g-95,g-96,g-97,g-98,g-99,g-100,g-101,g-102,g-103,g-104,g-105,g-106,g-107,g-108,g-109,g-110,g-111,g-112,g-113,g-114,g-115,g-116,g-117,g-118,g-119,g-120,g-121,g-122,g-123,g-124,g-125,g-126,g-127,g-128,g-129,g-130,g-131,g-132,g-133,g-134,g-135,g-136,g-137,g-138,g-139,g-140,g-141,g-142,g-143,g-144,g-145,g-146,g-147,g-148,g-149,g-150,g-151,g-152,g-153,g-154,g-155,g-156,g-157,g-158,g-159,g-160,g-161,g-162,g-163,g-164,g-165,g-166,g-167,g-168,g-169,g-170,g-171,g-172,g-173,g-174,g-175,g-176,g-177,g-178,g-179,g-180,g-181,g-182,g-183,g-184,g-185,g-186,g-187,g-188,g-189,g-190,g-191,g-192,g-193,g-194,g-195,g-196,g-197,g-198,g-199,g-200,g-201,g-202,g-203,g-204,g-205,g-206,g-207,g-208,g-209,g-210,g-211,g-212,g-213,g-214,g-215,g-216,g-217,g-218,g-219,g-220,g-221,g-222,g-223,g-224,g-225,g-226,g-227,g-228,g-229,g-230,g-231,g-232,g-233,g-234,g-235,g-236,g-237,g-238,g-239,g-240,g-241,g-242,g-243,g-244,g-245,g-246,g-247,g-248,g-249,g-250,g-251,g-252,g-253,g-254,g-255,g-256,g-257,g-258,g-259,g-260,g-261,g-262,g-263,g-264,g-265,g-266,g-267,g-268,g-269,g-270,g-271,g-272,g-273,g-274,g-275,g-276,g-277,g-278,g-279,g-280,g-281,g-282,g-283,g-284,g-285,g-286,g-287,g-288,g-289,g-290,g-291,g-292,g-293,g-294,g-295,g-296,g-297,g-298,g-299,g-300,g-301,g-302,g-303,g-304,g-305,g-306,g-307,g-308,g-309,g-310,g-311,g-312,g-313,g-314,g-315,g-316,g-317,g-318,g-319,g-320,g-321,g-322,g-323,g-324,g-325,g-326,g-327,g-328,g-329,g-330,g-331,g-332,g-333,g-334,g-335,g-336,g-337,g-338,g-339,g-340,g-341,g-342,g-343,g-344,g-345,g-346,g-347,g-348,g-349,g-350,g-351,g-352,g-353,g-354,g-355,g-356,g-357,g-358,g-359,g-360,g-361,g-362,g-363,g-364,g-365,g-366,g-367,g-368,g-369,g-370,g-371,g-372,g-373,g-374,g-375,g-376,g-377,g-378,g-379,g-380,g-381,g-382,g-383,g-384,g-385,g-386,g-387,g-388,g-389,g-390,g-391,g-392,g-393,g-394,g-395,g-396,g-397,g-398,g-399,g-400,g-401,g-402,g-403,g-404,g-405,g-406,g-407,g-408,g-409,g-410,g-411,g-412,g-413,g-414,g-415,g-416,g-417,g-418,g-419,g-420,g-421,g-422,g-423,g-424,g-425,g-426,g-427,g-428,g-429,g-430,g-431,g-432,g-433,g-434,g-435,g-436,g-437,g-438,g-439,g-440,g-441,g-442,g-443,g-444,g-445,g-446,g-447,g-448,g-449,g-450,g-451,g-452,g-453,g-454,g-455,g-456,g-457,g-458,g-459,g-460,g-461,g-462,g-463,g-464,g-465,g-466,g-467,g-468,g-469,g-470,g-471,g-472,g-473,g-474,g-475,g-476,g-477,g-478,g-479,g-480,g-481,g-482,g-483,g-484,g-485,g-486,g-487,g-488,g-489,g-490,g-491,g-492,g-493,g-494,g-495,g-496,g-497,g-498,g-499,g-500,g-501,g-502,g-503,g-504,g-505,g-506,g-507,g-508,g-509,g-510,g-511,g-512,g-513,g-514,g-515,g-516,g-517,g-518,g-519,g-520,g-521,g-522,g-523,g-524,g-525,g-526,g-527,g-528,g-529,g-530,g-531,g-532,g-533,g-534,g-535,g-536,g-537,g-538,g-539,g-540,g-541,g-542,g-543,g-544,g-545,g-546,g-547,g-548,g-549,g-550,g-551,g-552,g-553,g-554,g-555,g-556,g-557,g-558,g-559,g-560,g-561,g-562,g-563,g-564,g-565,g-566,g-567,g-568,g-569,g-570,g-571,g-572,g-573,g-574,g-575,g-576,g-577,g-578,g-579,g-580,g-581,g-582,g-583,g-584,g-585,g-586,g-587,g-588,g-589,g-590,g-591,g-592,g-593,g-594,g-595,g-596,g-597,g-598,g-599,g-600,g-601,g-602,g-603,g-604,g-605,g-606,g-607,g-608,g-609,g-610,g-611,g-612,g-613,g-614,g-615,g-616,g-617,g-618,g-619,g-620,g-621,g-622,g-623,g-624,g-625,g-626,g-627,g-628,g-629,g-630,g-631,g-632,g-633,g-634,g-635,g-636,g-637,g-638,g-639,g-640,g-641,g-642,g-643,g-644,g-645,g-646,g-647,g-648,g-649,g-650,g-651,g-652,g-653,g-654,g-655,g-656,g-657,g-658,g-659,g-660,g-661,g-662,g-663,g-664,g-665,g-666,g-667,g-668,g-669,g-670,g-671,g-672,g-673,g-674,g-675,g-676,g-677,g-678,g-67

### Feature selection

In [39]:
data = train_features.append(test_features)
data

sig_id cp_type  cp_time cp_dose       g-0       g-1       g-2  \
0     id_000644bb2  trt_cp       24      D1  1.135329  0.907871 -0.416056   
1     id_000779bfc  trt_cp       72      D1  0.119307  0.682082  0.272066   
2     id_000a6266a  trt_cp       48      D1  0.779911  0.946241  1.425076   
3     id_0015fd391  trt_cp       48      D1 -0.735068 -0.274041 -0.437981   
4     id_001626bd3  trt_cp       72      D2 -0.452153 -0.476994  0.972847   
...            ...     ...      ...     ...       ...       ...       ...   
3977  id_ff7004b87  trt_cp       24      D1  0.599228 -0.690841  2.410293   
3978  id_ff925dd0d  trt_cp       24      D1 -0.844647 -0.282627  2.017895   
3979  id_ffb710450  trt_cp       72      D1 -0.564750 -0.146369  0.233767   
3980  id_ffbb869f2  trt_cp       48      D2 -1.598704 -1.932100 -0.617743   
3981  id_ffd5800b6  trt_cp       72      D1 -0.740343  0.510587  0.308094   

           g-3       g-4       g-5       g-6       g-7       g-8       g-9  \
0    -0.968039 -0.255340 -1.015601 -1.366836 -0.025365  0.679176 -0.309849   
1     0.080090  1.204343  0.686856  0.314253  0.555638 -0.537128  0.838334   
2    -0.131449 -0.006720  1.494332  0.234189  0.365066 -0.005485  1.240305   
3     0.758995  2.450858 -0.858918 -2.302688  0.308319 -0.192281 -1.386051   
4     0.971264  1.464223 -0.870248 -0.375927 -0.204570 -1.064268  0.849390   
...        ...       ...       ...       ...       ...       ...       ...   
3977 -0.966991  1.073563  1.786479  0.093157 -0.777859 -1.302970  0.710507   
3978  0.507095  0.641807  0.434798 -1.008719  0.005065  0.316838  1.884350   
3979 -1.052821  0.066455  0.635301 -0.019257  1.183612  0.028173  1.396210   
3980  1.321004 -1.620956 -0.264244 -0.487322 -1.280867  1.176414  0.648341   
3981 -0.079867 -0.080527  1.098847 -1.177453  0.635053 -0.078526  0.241759   

          g-10      g-11      g-12      g-13      g-14      g-15      g-16  \
0     1.525286  0.170935  0.622365 -0.562485  0.280808 -1.057699 -1.141974   
1    -1.246925 -0.576240 -0.211084  0.545683  0.149631  0.390751  0.399016   
2    -0.642049 -0.770758 -0.130254 -2.318567  0.878104 -0.546810  0.471513   
3    -1.001516 -0.496024 -1.147069 -0.817017 -1.766285  1.393524 -0.228769   
4    -0.335974 -0.704161 -1.246515  1.183070  0.318119  0.342736 -0.025681   
...        ...       ...       ...       ...       ...       ...       ...   
3977  0.323337  0.953354  0.847746 -0.961399 -0.802193 -1.728311 -0.023723   
3978  0.222828  0.468210  0.943055 -1.360951  0.275127  2.272043  1.311850   
3979  0.578947 -0.586697 -0.242103 -0.387376  0.648839 -1.256767 -0.401755   
3980  1.863316  0.778689  1.243290  1.314189  1.392839  0.824837 -0.387557   
3981  0.270831  0.288072 -0.273702  1.371577 -0.648578  1.319761  0.494015   

          g-17      g-18      g-19      g-20      g-21      g-22      g-23  \
0     0.870306  0.380826 -0.510288 -0.723759 -1.330358 -1.677516  1.250091   
1     0.398856 -0.840190  0.499081  1.227475 -0.135705 -0.397160 -0.440854   
2    -0.406609 -0.306381 -0.042828 -0.105768 -1.010887 -1.966401  0.533838   
3    -1.053845  0.245322 -2.143516  2.100103 -1.350884 -1.101454 -1.060164   
4    -1.567247 -0.852875 -0.704816 -0.447090 -0.291741  1.054528  0.489594   
...        ...       ...       ...       ...       ...       ...       ...   
3977  0.382146  0.792974 -1.423784  1.281016 -0.953311  0.970004  1.505775   
3978 -0.248137  0.757823  0.885465  0.838818 -0.930381 -0.250864  0.691935   
3979  1.316144  0.963133 -1.226402 -1.346128 -0.164968 -0.312849  0.691935   
3980  0.708978  0.029391 -0.121104  1.221609 -1.790924  0.328457 -1.036820   
3981  0.588086 -1.151503 -1.862807  0.044387  0.464618 -1.265608  0.108212   

          g-24      g-25      g-26      g-27      g-28      g-29      g-30  \
0     0.559176  0.401912  0.240643  0.165332 -0.522229  0.771816  0.416076   
1     0.396454 -0.583305  0.632018  0.220515 -0.744947 -0.187210 -0.120986   
2     0.626802  0.698398 -1.379040  2.6845

In [40]:
var_thresh = VarianceThreshold(VarianceThreshold_for_FS)
data = train_features.append(test_features)
data_transformed = var_thresh.fit_transform(data.iloc[:, 4:])

train_features_transformed = data_transformed[ : train_features.shape[0]]
test_features_transformed = data_transformed[-test_features.shape[0] : ]


train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                              columns=['sig_id','cp_type','cp_time','cp_dose'])

train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed)], axis=1)


test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                             columns=['sig_id','cp_type','cp_time','cp_dose'])

test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed)], axis=1)

train_features.shape

(23814, 983)

In [41]:
train_features.head(5)

,sig_id,cp_type,cp_time,cp_dose,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978
0,id_000644bb2,trt_cp,24,D1,1.135329,0.907871,-0.416056,-0.968039,-0.255340,-1.015601,-1.366836,-0.025365,0.679176,-0.309849,1.525286,0.170935,0.622365,-0.562485,

In [42]:
train = train_features.merge(train_targets_scored, on='sig_id')
train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

target = train[train_targets_scored.columns]

In [43]:
train = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)

In [44]:
train.head(5)

,sig_id,cp_time,cp_dose,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosin

In [45]:
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()

### CV folds

In [46]:
folds = train.copy()

mskf = MultilabelStratifiedKFold(n_splits=7)

for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
    folds.loc[v_idx, 'kfold'] = int(f)

folds['kfold'] = folds['kfold'].astype(int)
folds

sig_id cp_time cp_dose         0         1         2         3  \
0      id_000644bb2      24      D1  1.135329  0.907871 -0.416056 -0.968039   
1      id_000779bfc      72      D1  0.119307  0.682082  0.272066  0.080090   
2      id_000a6266a      48      D1  0.779911  0.946241  1.425076 -0.131449   
3      id_0015fd391      48      D1 -0.735068 -0.274041 -0.437981  0.758995   
4      id_001626bd3      72      D2 -0.452153 -0.476994  0.972847  0.971264   
...             ...     ...     ...       ...       ...       ...       ...   
21943  id_fff8c2444      72      D1  0.237498 -1.230100  0.219031 -0.366167   
21944  id_fffb1ceed      24      D2  0.208495 -0.022136 -0.236379 -0.797301   
21945  id_fffb70c0c      24      D2 -1.910311  0.586229 -0.588718  1.296786   
21946  id_fffcb9e7c      24      D1  0.816563  0.417084  0.431959  0.300768   
21947  id_ffffdd77b      72      D1 -1.244018  1.569378 -0.268627  1.085885   

              4         5         6         7         8         9        10  \
0     -0.255340 -1.015601 -1.366836 -0.025365  0.679176 -0.309849  1.525286   
1      1.204343  0.686856  0.314253  0.555638 -0.537128  0.838334 -1.246925   
2     -0.006720  1.494332  0.234189  0.365066 -0.005485  1.240305 -0.642049   
3      2.450858 -0.858918 -2.302688  0.308319 -0.192281 -1.386051 -1.001516   
4      1.464223 -0.870248 -0.375927 -0.204570 -1.064268  0.849390 -0.335974   
...         ...       ...       ...       ...       ...       ...       ...   
21943 -0.329890  0.569039 -0.151299 -0.249377 -0.940954 -0.030719  0.703196   
21944 -0.675392  0.917995  0.737173  0.507993 -0.623786 -0.041841  0.293054   
21945 -1.003709  0.850604 -0.304687 -0.704853  0.629444  0.645803 -0.292201   
21946  1.071485 -0.023108  0.049803  0.099449  0.530589 -1.193376  0.734723   
21947 -0.511611 -2.108523 -1.629218  1.429255  2.282361  1.254978  2.104993   

             11        12        13        14        15        16        17  \
0      0.170935  0.622365 -0.562485  0.280808 -1.057699 -1.141974  0.870306   
1     -0.576240 -0.211084  0.545683  0.149631  0.390751  0.399016  0.398856   
2     -0.770758 -0.130254 -2.318567  0.878104 -0.546810  0.471513 -0.406609   
3     -0.496024 -1.147069 -0.817017 -1.766285  1.393524 -0.228769 -1.053845   
4     -0.704161 -1.246515  1.183070  0.318119  0.342736 -0.025681 -1.567247   
...         ...       ...       ...       ...       ...       ...       ...   
21943 -0.698886  0.447134  0.138846  1.110891 -0.313375  0.108114  0.277147   
21944  0.196910 -0.682243 -1.509573  0.379643 -0.763011  0.227242  0.227157   
21945  1.780664 -0.364732 -0.861345  1.167375  0.967810 -0.497108  0.184342   
21946  0.157676  1.704163  0.246641  0.783632 -2.244750 -0.004669  0.836996   
21947  1.286898  1.891905  1.620959 -0.752249  1.191279 -1.381115  0.031180   

             18        19        20        21        22        23        24  \
0      0.380826 -0.510288 -0.723759 -1.330358 -1.677516  1.250091  0.559176   
1     -0.840190  0.499081  1.227475 -0.135705 -0.397160 -0.440854  0.396454   
2     -0.306381 -0.042828 -0.105768 -1.010887 -1.966401  0.533838  0.626802   
3      0.245322 -2.143516  2.100103 -1.350884 -1.101454 -1.060164 -1.422587   
4     -0.852875 -0.704816 -0.447090 -0.291741  1.054528  0.489594  1.690332   
...         ...       ...       ...       ...       ...       ...       ...   
21943  0.053524  0.231275 -0.650470 -0.748782  0.739493 -0.691935  0.191677   
21944 -1.512780 -0.125211  1.124498 -0.508261  0.825976  0.534224 -0.699562   
21945  0.030898  0.189699 -0.291986 -1.120138  0.140274 -1.824105 -0.980705   
21946  0.433803  1.470173 -0.962249  1.380819  0.548125  2.920571  0.629901   
21947 -1.769763 -0.069813 -0.255239 -0.963117 -1.870427 -1.308144 -1.208618   

             25        26        27        28        29        30        31  \
0      0.401912  0.240643  0.165332 -0.522229  0.771816  0.416076 -1.101826   
1     -0.583305  0.632018  0.220515 -0.744947 -0.187210 -0.

In [47]:
print(train.shape)
print(folds.shape)
print(test.shape)
print(target.shape)
print(sample_submission.shape)

(21948, 1188)
(21948, 1189)
(3624, 982)
(21948, 207)
(3982, 207)


### Preprocessing

In [48]:
def process_data(data):
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    return data

In [49]:
feature_cols = [c for c in process_data(folds).columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['kfold','sig_id']]
len(feature_cols)

984

## Modeling

In [50]:
def run_multilabel_xgboost_k_fold(NFOLDS, seed):
    def objective(trial):
        params = {
            'objective': 'binary:logistic',
            'n_estimators': 1000,
            'learning_rate': 0.1,
            'subsample': trial.suggest_discrete_uniform('subsample', 0.6, 0.95, 0.05),
            'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.6, 0.95, 0.05),
            'max_depth':  trial.suggest_int('max_depth', 3, 9),
            'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
            'min_child_weight': trial.suggest_loguniform('min_child_weight', 0.1, 10.0),
            'tree_method': 'gpu_hist'
        }
        
#         params = {
#             'classify__estimator__objective': 'binary:logistic',
#             'classify__estimator__n_estimators': 1000,
#             'classify__estimator__learning_rate': 0.1,
#             'classify__estimator__subsample': trial.suggest_discrete_uniform('subsample', 0.6, 0.95, 0.05),
#             'classify__estimator__colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.6, 0.95, 0.05),
#             'classify__estimator__max_depth': trial.suggest_int('max_depth', 3, 9),
#             'classify__estimator__gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
#             'classify__estimator__min_child_weight': trial.suggest_loguniform('min_child_weight', 0.1, 10.0),
#         }
#         clf = Pipeline([('classify', MultiOutputClassifier(xgb.XGBClassifier(tree_method='gpu_hist')))])
#         clf.set_params(**params)
        
        cv_score = 0.            
        
        for fold in tqdm(range(NFOLDS)):
            seed_everything(seed)
            
            train = process_data(folds)
#             test_ = process_data(test)
            
            trn_idx = train[train['kfold'] != fold].index
            val_idx = train[train['kfold'] == fold].index

            train_df = train[train['kfold'] != fold].reset_index(drop=True)
            valid_df = train[train['kfold'] == fold].reset_index(drop=True)

            x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
            x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values

            clf = MultiOutputClassifier(xgb.XGBClassifier(**params))
            clf.fit(x_train, y_train)
#             clf.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], eval_metric='logloss', verbose=False, early_stopping_rounds=50)
            y_preds = clf.predict_proba(x_valid)

            score = 0.
            for i in range(len(target_cols)):
                y_pred = y_preds[i]
                score_ = log_loss(y_valid[:, i], y_pred[:, 1], labels=[0,1])
                score += score_ / target.shape[1]
            cv_score += score
        return cv_score / NFOLDS
    return objective

In [51]:
def run_multilabel_xgboost_bestparam(best_params, seed, submission):
    train_ = process_data(train)
    test_ = process_data(test)
    
    x_train, y_train = train_[feature_cols].values, train_[target_cols].values
    x_test = test_[feature_cols].values
    
    # train XGBoost by using best_params
    model = MultiOutputClassifier(xgb.XGBClassifier(**best_params))
    model.fit(x_train, y_train)
    
    # predict for test
    y_preds = model.predict_proba(x_test)
    
    for i in range(len(target_cols)):
        y_pred = y_preds[i]
        submission[traget_cols[i]] = y_pred[:, 1]
    
    # save model for test
    modelfile = 'BestModel4test_MultiXGBoost_SEED'+str(seed)+'.pkl'
    with open(os.path.join('result', 'best_model', modelfile), 'wb') as f:
        pickle.dump(model, f)
    

## Prediction & Submission

In [ ]:
# Averaging on multiple SEEDS

# SEED = [0, 1, 2, 3, 4, 5, 6]
SEED = [0, 1, 2]

for seed in tqdm(SEED):
    # optimize by optuna
    study = optuna.create_study()
    study.optimize(run_multilabel_xgboost_k_fold(NFOLDS, seed), n_trials=NUM_OPTUNA_TRIAL)
    
    # save optuna log
    df_trial = study.trials_dataframe()
    df_trial.to_pickle(os.path.join('report', 'OptunaLog_MultiXGBoost_SEED'+str(seed)+'.pkl'))
    
    # save best params found by CV
    bestparam_filename = 'BestParamsSelectedByCV_MultiXGBoost_SEED'+str(seed)+'.pkl'
    with open(os.path.join('result', 'best_param_cv', bestparam_filename), 'wb') as f:
        pickle.dump(study.best_params, f)
    

100%|████████████████████████████████████████████████████████████████████████████████| 2/2 [1:05:56<00:00, 1978.23s/it]
[I 2020-11-24 23:57:55,473] Finished trial#0 with value: 0.02066106134959387 with parameters: {'subsample': 0.65, 'colsample_bytree': 0.75, 'max_depth': 6, 'gamma': 0.12127146366891062, 'min_child_weight': 6.832538717180667}. Best is trial#0 with value: 0.02066106134959387.

 50%|█████████████████████████████████████████                                         | 1/2 [34:15<34:15, 2055.80s/it]

[Go to Top](#0)